In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd

In [70]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        nn.Module.__init__(self)
        self.l1 = nn.Linear(500, 1024)
        self.l2 = nn.Linear(1024, 256)
        self.l3 = nn.Linear(256, 32)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        return x
    
class SRPNet(nn.Module):
    def __init__(self, num_agents):
        super(SRPNet, self).__init__()
        self.num_agents = num_agents
        
        self.feature = Network()
        
        self.ll1 = nn.Linear(num_agents + 1, 32)
        self.rnn = nn.RNNCell(64, 32)
        self.ll2 = nn.Linear(32, 9)
        
    def forward(self, inp, vis, hid):
        feature = self.feature(inp)
        
        ll1_out = self.ll1(vis)
        rnn_inp = torch.cat([feature, ll1_out], dim=1)
        out = self.rnn(rnn_inp)
        ll2_out = self.ll2(out)
        
        return out, ll2_out

In [71]:
num_agents = 2
hid = torch.randn(8, 32)

stnet = SRPNet(num_agents)
for i in range(10):
    inp = torch.randn(8, 500)
    vis = torch.randn(8, num_agents+1)
    
    hid_out, ll2_out = stnet(inp, vis, hid)